In [38]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv('ETHERSCAN_API_KEY')

TARGET_PROXY_ADDRESS = ''

OFFSET = 100

url = "https://api.arbiscan.io/api"
params = {
    "module": "account",
    "action": "txlist",
    "address": TARGET_PROXY_ADDRESS,
    "startblock": 0,
    "endblock": "latest",
    "page": 1,
    "offset": OFFSET,
    "sort": "asc",
    "apikey": API_KEY
}

all_txs = []

while True:
    response = requests.get(url, params=params)
    txs = response.json()['result']
    txs_count = len(txs)
    if txs_count == 0:
        break
    print(f'found {txs_count} txs: in {txs[0]["blockNumber"]}..{txs[-1]["blockNumber"]}...')
    all_txs.extend(txs)
    params['page'] += 1

print(f"Total found {len(all_txs)} txs")


found 100 txs: in 119095598..132829749...
found 100 txs: in 132830013..154576856...
found 100 txs: in 154576923..180026033...
found 100 txs: in 180026092..267216960...
found 10 txs: in 267217964..273282893...
Total found 410 txs


In [41]:
# tx_type
TX_ERROR = "TX_ERROR"

CONTRACT_CREATION = "CONTRACT_CREATION"

# unpause()
UNPAUSE_SELECTOR = "0x3f4ba83a"
# pause()
PAUSE_SELECTOR = "0x8456cb59"
# diamondCut(tuple[] facetCuts,address target,bytes data)
DIAMOND_CUT_SELECTOR = "0x1f931c1c"

# https://eips.ethereum.org/EIPS/eip-2535
# enum FacetCutAction {Add, Replace, Remove}
# // Add=0, Replace=1, Remove=2
# struct FacetCut {
#     address facetAddress;
#     FacetCutAction action;
#     bytes4[] functionSelectors;
# }
# /// @notice Add/replace/remove any number of functions and optionally execute
# ///         a function with delegatecall
# /// @param _diamondCut Contains the facet addresses and function selectors
# /// @param _init The address of the contract or facet to execute _calldata
# /// @param _calldata A function call, including function selector and arguments
# ///                  _calldata is executed with delegatecall on _init
# function diamondCut(
#     FacetCut[] calldata _diamondCut,
#     address _init,
#     bytes calldata _calldata
# ) external;

import json
with open('diamond_cut_abi.json', 'r') as f:
    DIAMOND_CUT_ABI = json.load(f)

from web3 import Web3

w3 = Web3()

diamond_cut_contract = w3.eth.contract(abi=DIAMOND_CUT_ABI)

facet_to_selector_mp = {}
selector_to_facet_mp = {}

def analyze_tx(tx):
    if tx['isError'] != '0':
        return TX_ERROR
    
    if not tx['to']:
        return CONTRACT_CREATION

    if tx['methodId'] == DIAMOND_CUT_SELECTOR:
        _, args = diamond_cut_contract.decode_function_input(tx['input'])
        cuts = args['_diamondCut']
        for cut in cuts:
            action = cut['action']
            facet = cut['facetAddress']
            selectors = list(map(lambda x: x.hex(), cut['functionSelectors']))

            if action == 0:
                print(f'Add new cut: {facet} with selectors {selectors}')
                try:
                    facet_to_selector_mp[facet].extend(selectors)
                except KeyError:
                    facet_to_selector_mp[facet] = selectors
                
                for selector in selectors:
                    selector_to_facet_mp[selector] = facet

            elif action == 1:
                print(f'Replace cut: {facet} with selectors {selectors}')
                
                for selector in selectors:
                    old_facet = selector_to_facet_mp[selector]
                    facet_to_selector_mp[old_facet].remove(selector)
                    
                    try:
                        facet_to_selector_mp[facet].append(selector)
                    except KeyError:
                        facet_to_selector_mp[facet] = [selector]

                    selector_to_facet_mp[selector] = facet

            elif action == 2:
                print(f'Remove cut: {facet} with selectors {selectors}')

                for selector in selectors:
                    old_facet = selector_to_facet_mp[selector]
                    facet_to_selector_mp[old_facet].remove(selector)

                    selector_to_facet_mp.pop(selector)

for tx in all_txs:
    analyze_tx(tx)

def find_signautre(selector):
    try:
        response = requests.get(f'https://www.4byte.directory/api/v1/signatures/?hex_signature={selector}')
        count = response.json()['count']
        if count == 0:
            return None
        sig = response.json()['results'][0]['text_signature']
        if not sig:
            return None
    except Exception:
        return None
    
    return sig

import requests

def fetch_contract_info(address):
    params = {
        'module': 'contract',
        'action': 'getsourcecode',
        'address': address,
        'apikey': API_KEY
    }
    try:
        response = requests.get(url, params=params)
        res_json = response.json()
        name = res_json['result'][0]['ContractName']
        if not name:
            name = "unknown"
    except Exception:
        name = "unknown"

    return name

contract_name_mp = {}
facet_to_signatures_mp = {}
for selector, facet in selector_to_facet_mp.items():

    try:
        name = contract_name_mp[facet]
    except KeyError:
        name = fetch_contract_info(facet)
        contract_name_mp[facet] = name

    signature = find_signautre(selector)
    try:
        if not signature:
            signature = f"0x{selector}()"

        facet_to_signatures_mp[facet]['support_functions'].append(signature)
    except KeyError:
        facet_to_signatures_mp[facet] = {
            "contractName": name,
            "support_functions": [signature]
        }

data = {
    "facet_to_selectors": facet_to_selector_mp,
    "selctor_to_facet": selector_to_facet_mp,
    "facet_to_signatures": facet_to_signatures_mp
}

with open('facet_data.json', 'w') as f:
    json.dump(data, f)


Add new cut: 0x3E273bcc448d22b9329a56da3a91ace1B57e7D04 with selectors ['e1c7392a']
Add new cut: 0x69BeC70a49728544Fc402Ebb631Fb5B0b172b23F with selectors ['cdffacc6', '52ef6b2c', 'adfca15e', '7a0ed627', '01ffc9a7']
Add new cut: 0x8F1374641d3Cd7896c29E905346a9630260770B2 with selectors ['79ba5097', '8da5cb5b', '2f11d653', '21040fc3', 'd153b60c', '22ab8b4f']
Add new cut: 0x565Fd824c52014b8FC48Dd74508c256F4916C00A with selectors ['b1c9fbc1', 'e017e125']
Add new cut: 0xE4f74717fba81709439493Daf6DaD9a355d9Dbc8 with selectors ['3942a52e', 'a3aa8d1e', '14a6bf0f', 'ecd658b4', '3a264d10', 'e28a5ad3', 'd44e282b', '7914334f', '360398a3', 'd378d94e', '26ee4836', '8be3471a', 'a7ef7082', 'fd4344cd', '8f59b7a9', 'caa648b4', 'e9fb6299', '5ce23950', 'c3abc376']
Add new cut: 0x6dDB93d13EEf50950Fd27CcFb55a372027C6B34E with selectors ['9104c640']
Add new cut: 0x10D3DCA3b23d871A19C9B2fdeB6DdF076DE5ad3F with selectors ['79433d8b', '7614738f', 'd75c7462']
Add new cut: 0x5236b77da5836AeDCbEFB301e7F07B507d216